In [1]:
import cv2
import numpy as np
from work1 import estimate_dominant_colors, soft_segmentation, local_color_transfer, compute_region_features, match_regions
from work2 import pca_transfer, monge_kantorovitch_transfer

def hybrid_color_transfer(source_path, target_path, method="pca"):
    """
    Hybrid color transfer pipeline combining the best of work1.py and work2.py.
    """
    # Load images
    src_bgr = cv2.imread(source_path)
    tgt_bgr = cv2.imread(target_path)
    if src_bgr is None or tgt_bgr is None:
        raise IOError("Could not load source or target image.")

    # Step 1: Dominant Color Estimation
    src_dom = estimate_dominant_colors(src_bgr, grid_size=8, min_pixels=50)
    tgt_dom = estimate_dominant_colors(tgt_bgr, grid_size=8, min_pixels=50)

    # Step 2: Soft Segmentation
    src_seg = soft_segmentation(src_bgr, src_dom)  # shape(H,W,Cs)
    tgt_seg = soft_segmentation(tgt_bgr, tgt_dom)  # shape(H,W,Ct)

    # Ensure the same number of dominant colors in both images
    Cs = src_seg.shape[-1]
    Ct = tgt_seg.shape[-1]
    C_min = min(Cs, Ct)
    src_seg = src_seg[:, :, :C_min]
    tgt_seg = tgt_seg[:, :, :C_min]

    # Step 3: Global Color Alignment
    if method == "pca":
        aligned_bgr = pca_transfer(src_bgr, tgt_bgr)
    elif method == "monge-kantorovitch":
        aligned_bgr = monge_kantorovitch_transfer(src_bgr, tgt_bgr)
    else:
        raise ValueError("Invalid method. Choose 'pca' or 'monge-kantorovitch'.")

    # Step 4: Region Matching
    src_feats = compute_region_features(src_bgr, src_seg)
    tgt_feats = compute_region_features(tgt_bgr, tgt_seg)
    match_dict = match_regions(src_feats, tgt_feats, wS=1.0, wL=1.0, wR=1.0)

    # Step 5: Local Color Transfer
    out_lab = local_color_transfer(src_bgr, aligned_bgr, src_seg, tgt_seg, match_dict)
    out_bgr = cv2.cvtColor(out_lab, cv2.COLOR_LAB2BGR)

    return out_bgr

# Example Usage
if __name__ == "__main__":
    source_path = "/home/neelraj-reddy/college/6th_sem/computer vision/project/A little survey on previous works/images/beach_target.jpeg"
    target_path = "/home/neelraj-reddy/college/6th_sem/computer vision/project/A little survey on previous works/images/beach_ref.jpeg"

    # Run hybrid color transfer with PCA-based alignment
    result_bgr = hybrid_color_transfer(source_path, target_path, method="pca")
    cv2.imwrite("hybrid_result_pca.jpg", result_bgr)

    # Run hybrid color transfer with Monge-Kantorovitch alignment
    result_bgr = hybrid_color_transfer(source_path, target_path, method="monge-kantorovitch")
    cv2.imwrite("hybrid_result_mk.jpg", result_bgr)

/home/neelraj-reddy/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1920 but corresponding boolean dimension is 1994